# Cтатистики со шкалами
Пример отчета по расчету статистик со шкалами.

## Описание задачи

Посчитаем данные по интернет-ресурсу **Ivi**.

Общие параметры:
- Период: Сентябрь 2021
- География: Россия 0+
- Население: 12+
- Тип пользования интернетом: ограничения нет, считаем по всем (Web Desktop, Web Mobile, App Mobile)

Статистики:
- **Days Reached Frequency Distribution (drfd)** - количество человек, которые заходили на ресурс **Ivi** от 1 до 5 дней и от 10 до 20 дней.
- **Reach N+ Distribution (reachN)** - количество человек, которые заходили на ресурс **Ivi** от 2 до 10 раз и от 20 до 50 раз. 

# Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Cross Web и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.crossweb import tasks as cwt
from mediascope_api.crossweb import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Создаем объекты для работы с API Cross Web
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.CrossWebTask()
cats = cwc.CrossWebCats()

## Формируем задания

Для расчета статистик со шкалами обязательно нужно указать эти шкалы в переменной **scales**, а также прописать ее в параметрах, передаваемых в функцию **build_audience_task**.

### Общие параметры для заданий

Для начала зададим общие параметры

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец'). Можно указать несколько периодов
date_filter = [('2021-09-01', '2021-09-30')]

# Задаем фильтр по типам пользования интернетом
usetype_filter = [1,2,3]

### Получим ID ресурса
Для построения отчета необходимо получить идентификатор ресурса __Ivi__.

Для этого воспользуемся методом поиска в медиа справочнике. Работа с медиа справочником подробно описана в ноутбуке [catalogs](catalogs.ipynb). 

Получим идентификатор ресурса **Ivi**

In [ ]:
cats.get_resource(resource='Ivi')

Таким образом, необходимый идентификатор следующий:

- **Ivi**  resourceId = 1067

## Задания

Перейдем к формированию заданий.


### Задание №1. Расчет статистик со шкалами для ресурса Ivi

In [ ]:
# Задаем название ресурса для отображения в DataFrame
project_name = 'Ivi'

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = None

# Задаем фильтр по демографии, в нашем случае он не требуется
demo_filter = None

# Задаем фильтр по медиа, в нашем случае это ID ресурса Ivi
mart_filter = 'crossMediaResourceId = 1067'

# Указываем список срезов, чтобы сформировать структуру расчета
slices = ["researchMonth", "crossMediaResourceId"]

# Указываем список статистик для расчета
statistics = ['drfd', 'reachN']

# Так как указаны статистики drfd и reachN, зададим для них шкалы
scales = {'drfd':[(1, 5), (10, 20)], 
          'reachN':[(2, 10), (20, 50)]}

# Формируем задание для API Cross Web в формате JSON
task_json = mtask.build_task('media', project_name, date_filter, usetype_filter, geo_filter,
                             demo_filter, mart_filter, slices, statistics, scales)

# Отправляем задание на расчет и ждем выполнения
task_audience = mtask.wait_task(mtask.send_audience_task(task_json))

# Получаем результат
df_ivi = mtask.result2table(mtask.get_result(task_audience), project_name = project_name)
df_ivi

## Сохраняем в Excel

In [ ]:
writer = pd.ExcelWriter(mtask.get_excel_filename('audience-scales'))
df_info = mtask.get_report_info()
df_ivi.to_excel(writer, 'Report', index=False)
df_info.to_excel(writer, 'Info', index=False)
writer.save()